<div style="line-height:1.2;">

<h1 style="color:#BF66F2; margin-bottom: 0.3em;"> Convolutional Neural Networks in PyTorch 6 </h1>

<h4 style="margin-top: 0.3em; margin-bottom: 1em;"> Classification on various datasets. Focus on Binary-cross-entropy Loss. </h4>

<div style="line-height:1.4; margin-bottom: 0.5em;">
    <h3 style="color: lightblue; display: inline; margin-right: 0.5em;">Keywords:</h3> 
    cuda installation +  DataParallel (to use more than one GPU) + torch.cuda.device_count()
</div>

</div>

In [1]:
%%script echo Skipping, since already installed 
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.html
!pip install nvidia-pyindex
!apt-get update
!apt-get install cuda

Skipping, since already installed


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

In [3]:
%%script echo Skipping, since already installed
!nvidia-smi -L

Skipping, since already installed


In [4]:
class Net(nn.Module):
    """ Simple CNN Neural Network.\\
    1 input image channel, 6 output channels, 5x5 square convolution. 
    """
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)       # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        """ Perform a forward pass. 
        
        Details: 
        - Max pooling over a (2, 2) window.\\
        The pooling layer performs down-sampling on the feature maps produced by the convolutional layers. 
        - Flatten all dimensions except the batch dimension,\\
        the size of the flattened tensor is calculated based on the number of channels in the feature maps.
        """
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)          #if the size is a square, a single number is enough!
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
device = torch.device("cuda:0")
net.to(device)

In [7]:
#### Parameters
input_size = 5
output_size = 2
batch_size = 30
data_size = 100

In [8]:
class RandomDataset(Dataset):
    """ Custom PyTorch dataset class for generating random data.

    Args:
        - Size of each data sample [int]
        - Total number of data samples to generate [int]

    Attributes:
        - The length of the dataset, which is equal to the 'length' parameter [int]
        - Tensor containing randomly generated data samples [torch.Tensor]

    Methods:
        - __getitem__(self, index): Retrieves a data sample at the specified index.
        - __len__(self): Returns the total number of data samples in the dataset.

    """
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        """ Retrieve a data sample at the specified index. """
        return self.data[index]

    def __len__(self):
        """ Return the total number of data samples in the dataset. """
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size), batch_size=batch_size, shuffle=True)

In [9]:
class Model(nn.Module):
    """ Simple model of a Network with one fully-connected layer. """
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        """ Perform Forward Pass """
        output = self.fc(input)
        print("\tIn Model: input size", input.size(), "output size", output.size())

        return output

<h2 style="color:#BF66F2 ">  Main # 1</h2>

In [20]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

# Move the model to the GPU (or another specified device)
model.to(device) 

Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

In [21]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(), "output_size", output.size())

	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])


<h2 style="color:#BF66F2 ">  Main # 2</h2>

In [ ]:
""" Check for the availability of multiple GPUs and, if so, wrap your Net model with nn.DataParallel, 
to allow the usage of multiple GPUs for training and inference. """

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # Wrap the model with nn.DataParallel
    net = nn.DataParallel(net)

# Move the model to the GPU (or another specified device)
net.to(device) 

for data in rand_loader:
    input = data.to(device)
    output = net(input)
    print("Outside: input size", input.size(), "output_size", output.size())